In [ ]:
import numpy as np
import pickle

In [ ]:
BOARD_ROWS=3
BOARD_COLS=3

In [32]:
class Player:
    """
        defines simple AI player
        methods:
        - getHash(board) - returns string representation of board passed
        - availablePositions(current_board) - returns empty cells on board passed
        - get_action(current_board) - returns (i,j) formatted player turn
        - is_done(reward) - give player a revard
        - savepolicy() - saves policy for player, after end of the game
    """
    def __init__(self,name, symbol = 1, exp_rate=0.3):
        self.name = name
        self.symbol = symbol
        self.states = []  # record all positions taken
        self.lr = 0.2
        self.exp_rate = exp_rate
        self.decay_gamma = 0.9
        self.states_value = {}  # state -> value

    def getHash(self, board):
        boardHash = str(board.reshape(BOARD_COLS * BOARD_ROWS))
        return boardHash
    
    def availablePositions(self, board):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if board[i, j] == 0:
                    positions.append((i, j))  # need to be tuple
        return positions
    
    def get_action(self, current_board):
        positions = self.availablePositions(current_board)
        if np.random.uniform(0, 1) <= self.exp_rate:
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            value_max = -999
            for p in positions:
                next_board = current_board.copy()
                next_board[p] = self.symbol
                next_boardHash = self.getHash(next_board)
                value = 0 if self.states_value.get(next_boardHash) is None\
                        else self.states_value.get(next_boardHash)
                if value >= value_max:
                    value_max = value
                    action = p
        return action
    
    def addState(self, state):
        self.states.append(state)

    # at the end of game, backpropagate and update states value
    def is_done(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
            self.states_value[st] += self.lr * (self.decay_gamma * reward - self.states_value[st])
            reward = self.states_value[st]
        self.reset()

    def reset(self):
        self.states = []

    def savePolicy(self):

        fw = open('policy_' + str(self.name), 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()

    def loadPolicy(self, file):
        fr = open(file, 'rb')
        self.states_value = pickle.load(fr)
        fr.close()

In [35]:
class State:
    """
        game example class
        --
    """
    def __init__(self, p1, p2):

        self.board = np.zeros((BOARD_ROWS,BOARD_COLS))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        self.playerSymbol = 1

    # get unique hash of current board state
    def getHash(self):
        self.boardHash = str(self.board.reshape(BOARD_COLS * BOARD_ROWS))
        return self.boardHash

    def winner(self):
        # row
        for i in range(BOARD_ROWS):
            if sum(self.board[i, :]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[i, :]) == -3:
                self.isEnd = True
                return -1
        # col
        for i in range(BOARD_COLS):
            if sum(self.board[:, i]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[:, i]) == -3:
                self.isEnd = True
                return -1
        # diagonal
        diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
        diag_sum2 = sum([self.board[i, BOARD_COLS - i - 1] for i in range(BOARD_COLS)])
        diag_sum = max(abs(diag_sum1), abs(diag_sum2))
        if diag_sum == 3:
            self.isEnd = True
            if diag_sum1 == 3 or diag_sum2 == 3:
                return 1
            else:
                return -1

        # tie
        # no available positions
        if len(self.availablePositions()) == 0:
            self.isEnd = True
            return 0
        # not end
        self.isEnd = False
        return None

    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    positions.append((i, j))  # need to be tuple
        return positions

    def updateState(self, position):
        self.board[position] = self.playerSymbol
        # switch to another player
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1

    # only when game ends
    def giveReward(self):
        result = self.winner()
        # backpropagate reward
        if result == 1:
            print("P1 wins")
            self.showBoard()
            self.p1.is_done(1)
            self.p2.is_done(0)
        elif result == -1:
            print("P2 wins")
            self.showBoard()
            self.p1.is_done(0)
            self.p2.is_done(1)
        else:
            print("Draw")
            self.showBoard()
            self.p1.is_done(0.1)
            self.p2.is_done(0.5)

    # board reset
    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 1

    def play(self, rounds=100):
        for i in range(rounds):
            if i % 1000 == 0:
                print("Rounds {}".format(i))
            while not self.isEnd:

                # Player 1
                p1_action = self.p1.get_action(self.board)
                # take action and upate board state
                self.updateState(p1_action)
                board_hash = self.getHash()
                self.p1.addState(board_hash)
                # check board status if it is end

                win = self.winner()
                if win is not None:
                    # self.showBoard()
                    # ended with p1 either win or draw
                    self.giveReward()
                    self.p1.reset()
                    self.p2.reset()
                    self.reset()
                    break

                else:
                    # Player 2
                    p2_action = self.p2.get_action( self.board)
                    self.updateState(p2_action)
                    board_hash = self.getHash()
                    self.p2.addState(board_hash)

                    win = self.winner()
                    if win is not None:
                        # self.showBoard()
                        # ended with p2 either win or draw
                        self.giveReward()
                        self.p1.reset()
                        self.p2.reset()
                        self.reset()
                        break

    # play with human
    def play2(self):
        while not self.isEnd:
            # Player 1
            positions = self.availablePositions()
            p1_action = self.p1.get_action(positions, self.board)
            # take action and upate board state
            self.updateState(p1_action)
            self.showBoard()
            # check board status if it is end
            win = self.winner()
            if win is not None:
                if win == 1:
                    print(self.p1.name, "wins!")
                else:
                    print("tie!")
                self.reset()
                break

            else:
                # Player 2
                positions = self.availablePositions()
                p2_action = self.p2.get_action(positions)

                self.updateState(p2_action)
                self.showBoard()
                win = self.winner()
                if win is not None:
                    if win == -1:
                        print(self.p2.name, "wins!")
                    else:
                        print("tie!")
                    self.reset()
                    break

    def showBoard(self):
        # p1: x  p2: o
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('-------------')

In [36]:
# training
p1 = Player("p1")
p2 = Player("p2")

st = State(p1, p2)
print("Playing a few training rounds...")
st.play(10)
#store the success policies of each agent
p1.savePolicy()
p2.savePolicy()

Playing a few training rounds...
Rounds 0
P1 wins
-------------
| o | x | o | 
-------------
| o | x | x | 
-------------
| x | x | o | 
-------------
P1 wins
-------------
|   | x | x | 
-------------
|   | x | o | 
-------------
| x | o | o | 
-------------
P1 wins
-------------
| x | x | x | 
-------------
| o | x | o | 
-------------
| o | x | o | 
-------------
Draw
-------------
| o | x | x | 
-------------
| x | o | o | 
-------------
| x | o | x | 
-------------
P1 wins
-------------
| x |   | x | 
-------------
|   | x | o | 
-------------
| x | o | o | 
-------------
P2 wins
-------------
|   | x | o | 
-------------
| x | o | x | 
-------------
| o | x | o | 
-------------
P1 wins
-------------
| o |   | x | 
-------------
|   |   | x | 
-------------
|   | o | x | 
-------------
P1 wins
-------------
|   |   | x | 
-------------
|   |   | x | 
-------------
| o | o | x | 
-------------
P1 wins
-------------
| x | x | x | 
-------------
| o | x | o | 
-------------
| o | x |